Data Cleaning

In [54]:
import pandas as pd
import numpy as np
import os

os.chdir("E:\IntelliJ IDEA 2020.2\IdeaProjects\stock\data\source")

<>:5: SyntaxWarning: invalid escape sequence '\I'
<>:5: SyntaxWarning: invalid escape sequence '\I'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12340\3452757265.py:5: SyntaxWarning: invalid escape sequence '\I'
  os.chdir("E:\IntelliJ IDEA 2020.2\IdeaProjects\stock\data\source")


In [55]:
btc = pd.read_csv("bitcoin_2022-06_2025-06.csv",index_col=False)

btc.head()

,Price,Close,High,Low,Open,Volume,Prev Close,Weekday
0,Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,NaN,NaN
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-23,21085.876953125,21135.76171875,19950.1171875,19986.607421875,26188097173,NaN,3.0
3,2022-06-24,21231.65625,21472.91796875,20777.51171875,21084.6484375,24957784918,21085.876953,4.0
4,2022-06-25,21502.337890625,21520.9140625,20964.5859375,21233.609375,18372538715,21231.656250,5.0


In [56]:
btc = btc.iloc[2:,:]

btc = btc.astype({"Close":float, "High":float, "Low":float, 
                         "Open":float, "Volume":float, "Prev Close":float, "Weekday":int})

btc.head()

,Price,Close,High,Low,Open,Volume,Prev Close,Weekday
2,2022-06-23,21085.876953,21135.761719,19950.117188,19986.607422,2.618810e+10,NaN,3
3,2022-06-24,21231.656250,21472.917969,20777.511719,21084.648438,2.495778e+10,21085.876953,4
4,2022-06-25,21502.337891,21520.914062,20964.585938,21233.609375,1.837254e+10,21231.656250,5
5,2022-06-26,21027.294922,21783.724609,21016.269531,21496.494141,1.802717e+10,21502.337891,6
6,2022-06-27,20735.478516,21478.089844,20620.199219,21028.238281,2.096570e+10,21027.294922,0


In [57]:
btc["p_change"] = (btc["Open"] - btc["Close"])/btc["Open"]

# calculate average true range
"""
    14日平均真实波幅
"""

def cal_atr(r):
    return max(
        r['High'] - r['Low'],
        abs(r['High'] - r['Prev Close']) if pd.notnull(r['Prev Close']) else 0,
        abs(r['Low'] - r['Prev Close']) if pd.notnull(r['Prev Close']) else 0
    ) 

btc["TR"] = btc.apply(lambda x : cal_atr(x), axis=1)
btc["atr14"] = btc["TR"].rolling(window=14).mean()
btc["atr21"] = btc["TR"].rolling(window=21).mean()

btc.head()

,Price,Close,High,Low,Open,Volume,Prev Close,Weekday,p_change,TR,atr14,atr21
2,2022-06-23,21085.876953,21135.761719,19950.117188,19986.607422,2.618810e+10,NaN,3,-0.055000,1185.644531,NaN,NaN
3,2022-06-24,21231.656250,21472.917969,20777.511719,21084.648438,2.495778e+10,21085.876953,4,-0.006972,695.406250,NaN,NaN
4,2022-06-25,21502.337891,21520.914062,20964.585938,21233.609375,1.837254e+10,21231.656250,5,-0.012656,556.328125,NaN,NaN
5,2022-06-26,21027.294922,21783.724609,21016.269531,21496.494141,1.802717e+10,21502.337891,6,0.021827,767.455078,NaN,NaN
6,2022-06-27,20735.478516,21478.089844,20620.199219,21028.238281,2.096570e+10,21027.294922,0,0.013922,857.890625,NaN,NaN


In [58]:
# 10, 30 ma

btc['ma_short'] = btc['Close'].rolling(window=5).mean()
btc['ma_long'] = btc['Close'].rolling(window=20).mean()


In [59]:
short_window = 10
long_window = 50

btc["SMA_short"] = btc["Close"].rolling(short_window).mean()
btc["SMA_long"] = btc["Close"].rolling(long_window).mean()

In [60]:
# 2. 金叉 / 死叉 信号
btc['prev_ma_short'] = btc['ma_short'].shift(1)
btc['prev_ma_long'] = btc['ma_long'].shift(1)

btc['golden_cross'] = (btc['prev_ma_short'] <= btc['prev_ma_long']) & (btc['ma_short'] > btc['ma_long'])
btc['death_cross'] = (btc['prev_ma_short'] >= btc['prev_ma_long']) & (btc['ma_short'] < btc['ma_long'])

In [61]:
btc['ma_diff'] = btc['ma_short'] - btc['ma_long']
btc['ma_ratio'] = btc['ma_short'] / btc['ma_long']
btc['volatility'] = btc['High'] - btc['Low']

In [62]:
btc['future_return_1d'] = (btc['Close'].shift(-1) - btc['Close']) / btc['Close']

In [63]:
def return_to_score(r):
    if pd.isna(r):
        return np.nan
    # 截断在 ±10% 范围内，并映射为 0~100
    r = max(min(r, 0.10), -0.10)
    return round((r + 0.10) / 0.20 * 100)

btc['score'] = btc['future_return_1d'].apply(return_to_score)

In [64]:
btc.rename(columns={"Price":"Period"},inplace=True)

btc.head()

,Period,Close,High,Low,Open,Volume,Prev Close,Weekday,p_change,TR,...,SMA_long,prev_ma_short,prev_ma_long,golden_cross,death_cross,ma_diff,ma_ratio,volatility,future_return_1d,score
2,2022-06-23,21085.876953,21135.761719,19950.117188,19986.607422,2.618810e+10,NaN,3,-0.055000,1185.644531,...,NaN,NaN,NaN,False,False,NaN,NaN,1185.644531,0.006914,53.0
3,2022-06-24,21231.656250,21472.917969,20777.511719,21084.648438,2.495778e+10,21085.876953,4,-0.006972,695.406250,...,NaN,NaN,NaN,False,False,NaN,NaN,695.406250,0.012749,56.0
4,2022-06-25,21502.337891,21520.914062,20964.585938,21233.609375,1.837254e+10,21231.656250,5,-0.012656,556.328125,...,NaN,NaN,NaN,False,False,NaN,NaN,556.328125,-0.022093,39.0
5,2022-06-26,21027.294922,21783.724609,21016.269531,21496.494141,1.802717e+10,21502.337891,6,0.021827,767.455078,...,NaN,NaN,NaN,False,False,NaN,NaN,767.455078,-0.013878,43.0
6,2022-06-27,20735.478516,21478.089844,20620.199219,21028.238281,2.096570e+10,21027.294922,0,0.013922,857.890625,...,NaN,NaN,NaN,False,False,NaN,NaN,857.890625,-0.021936,39.0


In [65]:
btc.shape

(1096, 25)

In [66]:
btc.to_csv("cleaned_btc_2022-06_2025-06.csv")